In [ ]:
%matplotlib inline
from collections import Counter
from collections import defaultdict
import scanpy as sc
import scrublet as scr
import pandas as pd
import pickle as pkl
import numpy as np
from bbknn import bbknn
import scipy
import matplotlib.pyplot as plt
import re
import glob
import os
import sys
from geosketch import gs
from numpy import cov
import scipy.cluster.hierarchy as spc
import seaborn as sns; sns.set(color_codes=True)
from sklearn.linear_model import LogisticRegression
import sklearn
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=80, color_map='viridis')
sc.logging.print_versions()

In [ ]:
#Introduce variables

#name of first object (arbitrary)
data1 = "_Training"
Object1 = './overall_data/Healthy_all_data.h5ad'
#provide categorical containing donor information
donor_id = 'donor_id'
#provide cateorical to join between datasets, this shoould be annotations for cells (obs col)
cat1 = 'final'
#provide an output path and a folder name to be created
output = "./logit_regression_out_v2/"

## Start of processing module

In [ ]:
# Load dataset 1
adata_orig = sc.read(Object1)

In [ ]:
adata_orig.obs.columns

In [ ]:
#Get some basic processing done for the orig data
sc.pp.neighbors(adata_orig,n_neighbors=15, n_pcs=50)
sc.external.pp.bbknn(adata_orig, batch_key='donor_id', approx=True, metric='angular', copy=False, n_pcs=50, trim=None, n_trees=10, use_faiss=True, set_op_mix_ratio=1.0, local_connectivity=1)
sc.tl.umap(adata_orig)
sc.pl.umap(adata_orig,color = 'final')
######

# LR function

In [ ]:
def LR_compare(adata, train_x ,train_label,subset_predict, subset_train,penalty='l2',sparcity=0.2,col_name='predicted'):
    #Define LR parameters
    penalty='l2'
    sparcity=0.2
    lr = LogisticRegression(penalty=penalty, C=sparcity)

    if train_x == 'X':
        train_label = adata.obs[common_cat].values
        train_label = train_label[subset_train]
        train_x = adata.X
        predict_x = train_x
        train_x = train_x[subset_train, :]
        predict_x = train_x
        predict_x = predict_x[subset_predict]


    elif train_x in adata.obsm.keys():
        #Define training parameters
        train_label = adata.obs[common_cat].values
        train_label = train_label[subset_train]
        train_x = adata.obsm[train_x]
        predict_x = train_x
        train_x = train_x[subset_train, :]

        #Define Prediction parameters
        predict_x = predict_x[subset_predict]
        predict_x = pd.DataFrame(predict_x)
        predict_x.index = adata.obs[subset_predict].index

        
    #Train predictive model
    model = lr.fit(train_x, train_label)
    lr.fit(train_x, train_label)
    predict = lr.predict_proba(predict_x)

    #Create prediction table and map to adata.obs
    predict = lr.predict(predict_x)
    predict = pd.DataFrame(predict)
    predict.index = adata.obs[subset_predict].index
    adata.obs[col_name] = adata.obs.index
    adata.obs[col_name] = adata.obs[col_name].map(predict[0])

# Start of predictive and output loop

In [ ]:
#Create output directory
if os.path.exists(output) == True:
    print("Path already exists!")
else:
    os.mkdir(output)

os.chdir('./'+output)
print(os.getcwd())


#Start of the subset and plotting loop
for i in adata_orig.obs[donor_id].unique():
#Create an on-memory copy of the original data for the loop
    adata = adata_orig[:]
    adata2 = adata[:]
    adata2 = adata2[~adata2.obs[donor_id].isin([i])]
#name of second object
    data2 = "_prediction"
#provide cateorical to join between datasets
    cat2 = cat1
    
#create a common obs column in both datasets containing the data origin tag
    common_cat = "corr_concat" 
    adata.obs[common_cat] = adata.obs[cat1].astype(str) + data1
    adata2.obs[common_cat] = adata2.obs[cat2].astype(str) + data2
    adata.obs = adata.obs.astype('category')
    adata2.obs = adata2.obs.astype('category')
    #adata.raw = adata
    #adata2.raw = adata2

#concat the data
    concat = adata.concatenate(adata2, join='inner', index_unique='-', batch_key='Status')
    adata = concat
    print('data_concatenated!')
#Get PCA by covarainces of variable genes
    sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
    sc.pp.log1p(adata)
    #sc.pp.highly_variable_genes(adata, min_mean=0.1, max_mean=4)

#%% PCA
    sc.pp.pca(adata, n_comps=15, use_highly_variable=False, svd_solver='arpack')
    
#Define the seprator category in the column of interest, this works by partial matches and enables a-symmetric comparisons
    Data1_group = data1
    Data2_group = data2

#Define the common .obs column between cancatinated data
    common_cat = "corr_concat"

#Define the resource to train and predict on, PCA or X or UMAP (#if you wish to use gene expression, train_x = 'X' or 'X_pca' , or 'X_umap'
    #train_x = 'X'
    train_x = 'X_pca'
    ########################################################################################
    group1 = (adata.obs[common_cat][adata.obs[common_cat].str.contains(Data1_group)]).unique()
    group1 = list(group1)
    group2 = (adata.obs[common_cat][adata.obs[common_cat].str.contains(Data2_group)]).unique()
    group2 = list(group2)
    subset_predict = np.array(adata.obs[common_cat].isin(group2))
    subset_train = np.array(adata.obs[common_cat].isin(group1))
    train_label = adata.obs[common_cat].values
    #########################################################################################
    
#Run LR
    LR_compare(adata, train_x,train_label,subset_predict, subset_train,penalty='l2',sparcity=0.2,col_name='predicted')
    print('LR_predictions completed for loop ' + i + '!' )
#Plot predictions in probability heatmap
    x='predicted'
    y = common_cat
    y_attr = adata.obs[y]
    x_attr = adata.obs[x]
    crs_tbl = pd.crosstab(x_attr, y_attr)
    #crs_tbl_test = crs_tbl
    
#Cross table removes all result which have 0 matches, add these back into table
    vals = list(crs_tbl.index)
    pred = pd.DataFrame(group1)
    missing_vals = pred.iloc[:,0][~(pred.iloc[:,0].isin(vals))]
    crs_tbl = crs_tbl.T
    for missing in missing_vals:
        crs_tbl[missing] = 0
    crs_tbl = crs_tbl.reindex(sorted(crs_tbl.columns), axis=1)
    crs_tbl = crs_tbl.T
    crs_tbl = crs_tbl.reindex(sorted(crs_tbl.columns), axis=1)
    for col in crs_tbl :
        crs_tbl[col] = crs_tbl[col].div(crs_tbl[col].sum(axis=0)).multiply(100)

#Optionally save this image!
    #plot_df_heatmap(crs_tbl.T, cmap='coolwarm', rotation=90, figsize=figsize, vmin=20, vmax=70)
    #pal = sns.diverging_palette(240, 10, n=10)
    #g = sns.clustermap(crs_tbl.T, cmap=pal,vmin=20, vmax=70,linewidths=.5)
    #g = sns.heatmap(crs_tbl, cmap=pal, vmin=0, vmax=100,linewidths=.5 ,center=50,square=True )
    #plt.show();
    
#Save probability distribution
    prob_out_name = "./logist_prediction_prob_donor_minus_" + i+ ".csv"
    crs_tbl.to_csv(prob_out_name)
    print("LR probs saved for loop " + i +"!")
############################

    #Crete a holder for current obj
    adata3 = adata[:]
    #load original preiction object
    adata_predicted = adata2[:]
    #Assign matches
    adata3 = adata3[(adata3.obs[common_cat].isin(group2))]
    adata3.obs[common_cat].unique()
    adata_predicted.obs['predicted'] = adata_predicted.obs.index
    adata3.obs.index = adata_predicted.obs.index
    adata3.obs[common_cat].astype(str)
    adata_predicted.obs['predicted'] = adata3.obs["predicted"].astype(str)
    
#Frequency redistribution by additive assignment if reclustering is done
    cluster_prediction = "clus_prediction"
    clusters_reassign = "leiden_res5"
    res= 5
    lr_predicted_col = 'predicted'
    
##Plot Umap with Rand index and mutual info score
    sc.pp.neighbors(adata_predicted,n_neighbors=15, n_pcs=50)
    sc.pp.highly_variable_genes(adata_predicted, min_mean=0.1, max_mean=4)
    sc.pp.pca(adata_predicted, n_comps=15, use_highly_variable=False, svd_solver='arpack')
    sc.external.pp.bbknn(adata_predicted, batch_key='donor_id', approx=True, metric='angular', copy=False, n_pcs=15, trim=None, n_trees=10, use_faiss=True, set_op_mix_ratio=1.0, local_connectivity=1)
    sc.tl.umap(adata_predicted)

    sc.tl.leiden(adata_predicted, resolution= res, key_added= clusters_reassign, random_state=24, n_iterations=-1)
    adata_predicted.obs[cluster_prediction] = adata_predicted.obs.index
    for z in adata_predicted.obs[clusters_reassign].unique():
        df = adata_predicted.obs
        df = df[(df[clusters_reassign].isin([z]))]
        df_count = pd.DataFrame(df[lr_predicted_col].value_counts())
        freq_arranged = df_count.index
        cat = freq_arranged[0]
        df.loc[:,cluster_prediction] = cat
        adata_predicted.obs.loc[adata_predicted.obs[clusters_reassign] == z, [cluster_prediction]] = cat
    #adata_predicted.obs[cluster_prediction] = adata_predicted.obs['predicted']
#caculate Rand and MI
    rand=sklearn.metrics.adjusted_rand_score(list(adata_predicted.obs[cat1]), list(adata_predicted.obs[cluster_prediction]))
    mi=sklearn.metrics.adjusted_mutual_info_score(list(adata_predicted.obs[cat1]), list(adata_predicted.obs[cluster_prediction]), average_method='arithmetic')
#include info in plot title
##Edit the predicted col and get all missing values
    adata_predicted.obs[cluster_prediction] = adata_predicted.obs[cluster_prediction].str.replace('_Training', '', regex=True)
    adata_predicted.obs[cluster_prediction]
    fig_name = str(i) + " Adj_Rnd= " + str(rand) + " Mut_info= " + str(mi)
    sc.pl.umap(adata_predicted,color=cluster_prediction,title = fig_name,save=("_"+str(i)+".png"))
    print("Umap, rand,Mi plotted for loop " + i + "!")
# prints the missing and additional elements in list2 into our dataframe
    missing = set(list(adata_predicted.obs[cat1])).difference(list(adata_predicted.obs[cluster_prediction]))

##Create data frame and populate with scoring metrics
    temp_score = pd.DataFrame(columns=[i],dtype=object)
    temp_score = pd.DataFrame(temp_score.T)
    temp_score["adj_rand"] = rand
    temp_score["mutual_info"] = mi
    temp_score["missing_vals"] = "NAN"
    temp_score.at[i, "missing_vals"] = missing

    if 'concat_score' in globals():
        concat_score = pd.concat([concat_score, temp_score])
    else:
        concat_score = temp_score[:]
    
#Save file?
    #save_file_predicted = "./adata_predicted_donor_minus_" + i
    #adata.write(save_file_orig)
    
#Return resources back to system and restart the loop
    #gc.collect()
    del adata2
    del adata3
    print("end of loop!")
#Write out overall scored output
concat_score.to_csv("./concatenated_scores_minus_donor.csv")